In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import scipy.special as sp
from scipy.optimize import curve_fit

#import pandas as pd

In [ ]:
# Connect to a camera and an SLM

#cam.close()
#slm.close()

### Camera
from slmsuite.hardware.cameras.thorlabs import ThorCam
ThorCam.info(verbose=True)

# Serial number is 23910 for the Zelux, apparently.
cam = ThorCam(serial="23910", verbose=True, fliplr=False) 

### SLM 
from slmsuite.hardware.slms.screenmirrored import ScreenMirrored
ScreenMirrored.info(verbose=True)
slm = ScreenMirrored(display_number = 1, wav_um=.589, wav_design_um = .700)

### Make a composite optical system -- a Fourier pair.
from slmsuite.hardware.cameraslms import FourierSLM
fs = FourierSLM(cam, slm)
#fs.load_wavefront_calibration(plot=True)

In [ ]:
# Perform Fourier Calibration

cam.set_exposure(0.012)
fs.fourier_calibrate(array_shape=[5, 5], array_pitch=[30, 30], plot=True)

cam.set_exposure(.002) # Restore exposure

In [ ]:
# Create a simple array

pitch = 20 # pixels

xlist = np.arange(1000, 1060, pitch)
ylist = np.arange(1000, 1060, pitch)
xgrid, ygrid = np.meshgrid(xlist, ylist)
square = np.vstack((xgrid.ravel(), ygrid.ravel()))

plt.scatter(square[0, :], square[1, :], s=2)
plt.xlim([0, fs.cam.shape[1]]); plt.ylim([fs.cam.shape[0], 0])
plt.show()
print(xlist)
print(ylist)

In [ ]:
# Ceate the hologram

from slmsuite.holography.algorithms import SpotHologram
hologram = SpotHologram(shape=(2048, 2048), spot_vectors=square, basis="ij", cameraslm=fs)

hologram.plot_farfield(hologram.target,  limits=[[800, 1200], [800, 1200]], title="Target")

In [ ]:
# Calculate Phase Pattern

hologram.optimize('WGS-Kim', feedback='computational_spot', stat_groups=['computational_spot'], maxiter=50)
hologram.plot_farfield(limits=[[800, 1200], [800, 1200]], title="After Optimization")

In [ ]:
# Write to the SLM and get camera image

# Experimentally:
fs.slm.write(hologram.extract_phase(), settle=True)             # Write hologram.
fs.cam.set_exposure(.0015)
fs.cam.flush()
img = fs.cam.get_image()                                        # Grab image.

plt.figure(figsize=(24,24));    plt.imshow(img, vmax=50)        # This is saturated for visibility.
plt.xlabel("Image $x$ [pix]");  plt.ylabel("Image $y$ [pix]")
plt.show()

In [ ]:
# Create bounding boxes

